In [1]:
import numpy as np
from numpy import random as rnd
import pandas as pd
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [2]:
# auxiliary function that helps 
def pit(it, *pargs, **nargs):
    import enlighten
    global __pit_man__
    try:
        __pit_man__
    except NameError:
        __pit_man__ = enlighten.get_manager()
    man = __pit_man__
    try:
        it_len = len(it)
    except:
        it_len = None
    try:
        ctr = None
        for i, e in enumerate(it):
            if i == 0:
                ctr = man.counter(*pargs, **{**dict(leave = False, total = it_len), **nargs})
            yield e
            ctr.update()
    finally:
        if ctr is not None:
            ctr.close()

In [ ]:


def LS_put(S,K,T,r,σ,M,N,poly_degree):
    Δt = T/M
    discount_factor = np.exp(-r * Δt)
    Z = rnd.normal(size=(int(N/2),M))
    Z = np.concatenate((Z,-Z))
    S_t = np.zeros((N,M+1)) # create matrix for all paths
    S_t[:,0] = S # intitialize S_0 for all paths
    a1 = (r - .5*σ**2)*Δt # constant calculated outside of loop to reduce flops required
    a2 = σ*np.sqrt(Δt) # constant calculated outside of loop to reduce flops required
    for k in range(M):
        S_t[:,k+1] = S_t[:,k] * np.exp(a1 + a2*Z[:,k])
        
    payoff = np.maximum(K - S_t, np.zeros_like(S_t))
    value = np.zeros_like(payoff)
    value[:, -1] = payoff[:, -1]
    for t in tqdm(range(M-1 , 0 , -1)):
        regression = np.polyfit(S_t[:, t], 
                                value[:, t + 1]*discount_factor,
                                poly_degree)
        continuation_value = np.polyval(regression, S_t[:, t])
        value[:, t] = np.where(
            payoff[:, t] > continuation_value,
            payoff[:, t],
            value[:, t + 1] * discount_factor)
    option_premium = np.mean(value[:, 1] * discount_factor)
    std_err = np.std(value[:, 1] * discount_factor)/np.sqrt(N)
    return option_premium, std_err

In [ ]:
S_array = [36,38,40,42,44]
σ_array = [.2,.4]
T_array = [1,2]
K = 40
r = .06
m = 50
N = 100000
poly_degree = 6

LS_px = [4.472,4.821,7.091,8.488,
         3.244,3.735,6.139,7.669,
         2.313,2.879,5.308,6.921,
         1.617,2.206,4.588,6.243,
         1.118,1.675,3.957,5.622]

LS_df = pd.DataFrame({'S': pd.Series(dtype='int'),
                        'σ': pd.Series(dtype='float'),
                        'T': pd.Series(dtype='int'),
                        'LS Price': pd.Series(dtype='float'),
                        'My Price': pd.Series(dtype='float'),
                        'Std. Err': pd.Series(dtype='float')})

i = 0
for S in S_array:
    for σ in σ_array:
        for T in T_array:
            M = int(m*T)
            px, err = LS_put(S,K,T,r,σ,M,N,poly_degree)
            LS_df.loc[len(LS_df)] = [S,σ,T,LS_px[i],px,err]
            i += 1
            
LS_df['Difference from Paper'] = LS_df['My Price'] - LS_df['LS Price']
            
print(LS_df.to_latex(index=False))

In [3]:
def RF_put(S,K,T,r,σ,M,N,antithetic,tree_num,depth,min_samples,samples_per_tree,):
    Δt = T/M
    discount_factor = np.exp(-r * Δt)
    if antithetic == False:
        Z = rnd.normal(size=(N,M))
        Z2 = rnd.normal(size=(N,M))
    else:
        Z = rnd.normal(size=(int(N/2),M))
        Z = np.concatenate((Z,-Z))
        Z2 = rnd.normal(size=(int(N/2),M))
        Z2 = np.concatenate((Z2,-Z2))
    S_t = np.zeros((N,M+1)) # create matrix for all paths
    S_t[:,0] = S # intitialize S_0 for all paths
    S_t2 = np.zeros((N,M+1)) # create matrix for all paths
    S_t2[:,0] = S # intitialize S_0 for all paths
    a1 = (r - .5*σ**2)*Δt # constant calculated outside of loop to reduce flops required
    a2 = σ*np.sqrt(Δt) # constant calculated outside of loop to reduce flops required
    for k in range(M):
        S_t[:,k+1] = S_t[:,k] * np.exp(a1 + a2*Z[:,k])    
        S_t2[:,k+1] = S_t2[:,k] * np.exp(a1 + a2*Z2[:,k]) 
    payoff = np.maximum(K - S_t, np.zeros_like(S_t))
    payoff2 = np.maximum(K - S_t2, np.zeros_like(S_t2))
    value = np.zeros_like(payoff)
    value[:, -1] = payoff[:, -1]
    value2 = np.zeros_like(payoff2)
    value2[:, -1] = payoff2[:, -1]
    rf = RandomForestRegressor(n_estimators=tree_num, 
                               max_depth=depth, 
                               min_samples_leaf=min_samples, 
                               bootstrap=True, 
                               n_jobs=-1, 
                               max_samples=samples_per_tree)

    for t in pit(range(M-1 , 0 , -1)):
        X = S_t[:,t].reshape(-1,1)
        Y = value[:,t+1]
        rf.fit(X,Y)
        continuation_value = rf.predict(X)
        value[:, t] = np.where(
            payoff[:, t] > continuation_value,
            payoff[:, t],
            value[:, t + 1] * discount_factor)
        continuation_value2 = rf.predict(S_t2[:,t].reshape(-1,1))
        value2[:, t] = np.where(
            payoff2[:, t] > continuation_value2,
            payoff2[:, t],
            value2[:, t + 1] * discount_factor)
    option_premium = np.mean(value2[:, 1] * discount_factor)
    std_err = np.std(value2[:, 1] * discount_factor)/np.sqrt(N)
    return option_premium, std_err

In [4]:
# French paper (bermudan) px = 11.987
S = 100
K = 110
T = 1
r = .1
σ = .25
m = 10
M = int(m*T)
N = 100000
tree_num = 10
antithetic = True

RF_df = pd.DataFrame({'max_depth': pd.Series(dtype='int'),
                        'min_samples_leaf': pd.Series(dtype='int'),           
                        'max_samples': pd.Series(dtype='float'),
                        'RF Price': pd.Series(dtype='float'),
                        'Error': pd.Series(dtype='float'),
                        'Std. Err': pd.Series(dtype='float')})

depth_array = [3,5,10,20,100]
min_samples_leaf_array = [50,100,150,200]
max_samples_array = [1/tree_num,.2,.3,.4,.5]

for depth in pit(depth_array):
    for min_samples in pit(min_samples_leaf_array):
        for max_samples in pit(max_samples_array):
            px, std_err = RF_put(S,K,T,r,σ,M,N,antithetic,tree_num,depth,min_samples,max_samples)
            RF_df.loc[len(RF_df)] = [depth,min_samples,max_samples,px,px-11.987,std_err]
RF_df.to_pickle('./RF_df.pkl')

In [6]:
xscale = 3
yscale = 2
f, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2,sharey=True,figsize=(6.4*yscale,4.8*xscale))
ax6.remove()

f.suptitle('Hyperparameter Performance over 100,000 simulated paths', fontsize=18)
depth = depth_array[0]
ax1.set_title('Max tree depth={}'.format(depth))
for i in range(len(min_samples_leaf_array)):
    ax1.plot(max_samples_array,
             RF_df[(RF_df['max_depth'] == depth) & (RF_df['min_samples_leaf'] == min_samples_leaf_array[i])]['Error'],
             label='min_samples_leaf = {}'.format(min_samples_leaf_array[i]))
ax1.set_ylabel('Error')
ax1.legend()

depth = depth_array[1]
ax2.set_title('Max tree depth={}'.format(depth))
for i in range(len(min_samples_leaf_array)):
    ax2.plot(max_samples_array,
             RF_df[(RF_df['max_depth'] == depth) & (RF_df['min_samples_leaf'] == min_samples_leaf_array[i])]['Error'],
             label='min_samples_leaf = {}'.format(min_samples_leaf_array[i]))
ax2.legend()

depth = depth_array[2]
ax3.set_title('Max tree depth={}'.format(depth))
for i in range(len(min_samples_leaf_array)):
    ax3.plot(max_samples_array,
             RF_df[(RF_df['max_depth'] == depth) & (RF_df['min_samples_leaf'] == min_samples_leaf_array[i])]['Error'],
             label='min_samples_leaf = {}'.format(min_samples_leaf_array[i]))
ax3.set_ylabel('Error')
ax3.legend()

depth = depth_array[3]
ax4.set_title('Max tree depth={}'.format(depth))
for i in range(len(min_samples_leaf_array)):
    ax4.plot(max_samples_array,
             RF_df[(RF_df['max_depth'] == depth) & (RF_df['min_samples_leaf'] == min_samples_leaf_array[i])]['Error'],
             label='min_samples_leaf = {}'.format(min_samples_leaf_array[i]))
ax4.set_xlabel('% of dataset allocated to each tree',fontsize=12)
ax4.legend()

depth = depth_array[4]
ax5.set_title('Max tree depth={}'.format(depth))
for i in range(len(min_samples_leaf_array)):
    ax5.plot(max_samples_array,
             RF_df[(RF_df['max_depth'] == depth) & (RF_df['min_samples_leaf'] == min_samples_leaf_array[i])]['Error'],
             label='min_samples_leaf = {}'.format(min_samples_leaf_array[i]))
ax5.set_ylabel('Error')
ax5.set_xlabel('% of dataset allocated to each tree',fontsize=12)
ax5.legend()
plt.show()
